In [36]:
# Node class to track each node and its risk etc.
import math
class Node:
    def __init__(self, risk, xCoord, yCoord):
        self.risk = risk
        self.xCoord = xCoord
        self.yCoord = yCoord
        self.visited = False
        self.neighbors = []
        if xCoord == 0 and yCoord == 0:
            self.totalRisk = 0
        else:
            self.totalRisk = math.inf
    
    def addNeighbor(self, neighbor):
        self.neighbors.append(neighbor)
    
    def getCoordinates(self):
        return (self.xCoord, self.yCoord)

    def visit(self):
        self.visited = True

    def updateTotalRisk(self, newTotalRisk):
        self.totalRisk = newTotalRisk

In [37]:
# For each point on the map, create an instance of the Node class
def build_nodes(map):
    # Create a dictionary of nodes - key: (xCoordinate, yCoordinate), value: Node
    nodes = {}
    # Iterate through the map
    for rowIdx, row in enumerate(map):
        for colIdx, risk in enumerate(row):
            # For each point on the map, create a new Node
            newNode = Node(risk, rowIdx, colIdx)
            nodes.update({(rowIdx, colIdx): newNode})
    # For each node, create its list of neighbors
    for node in nodes.values():
        (xCoord, yCoord) = node.getCoordinates()
        up = nodes.get((xCoord - 1, yCoord))
        down = nodes.get((xCoord + 1, yCoord))
        left = nodes.get((xCoord, yCoord - 1))
        right = nodes.get((xCoord, yCoord + 1))
        if up:
            node.addNeighbor(up)
        if down:
            node.addNeighbor(down)
        if left:
            node.addNeighbor(left)
        if right:
            node.addNeighbor(right)
    return nodes


In [38]:
import heapq
# Solve part 1 by using Djikstra's
def solve(nodes):
    # Build a priority queue and initialize it with the node in (0,0)
    priorityQueue = []
    startingNode = nodes.get((0,0))
    heapq.heappush(priorityQueue, (startingNode.totalRisk, startingNode.getCoordinates(), startingNode))

    # While there's something in the priority queue, keep iterating
    while priorityQueue:
        # Pop the item with the lowest total risk (break ties based on coordinates - doesn't matter, but need a tiebreaker so it doesn't crash)
        _, _, currentNode = heapq.heappop(priorityQueue)
        # Marke the current node visited
        currentNode.visit()

        # Iterate through all of the current node's neighbors
        for neighbor in currentNode.neighbors:
            # Skip if already visited
            if neighbor.visited: continue

            # Otherwise, update the neighbor's total risk if we've found a better path than exists
            newTotalRisk = neighbor.risk + currentNode.totalRisk
            if newTotalRisk < neighbor.totalRisk:
                neighbor.updateTotalRisk(newTotalRisk)
                # Adding the coordinates in here to add another layer of comparison to avoid bug
                heapq.heappush(priorityQueue, (newTotalRisk, neighbor.getCoordinates(), neighbor))

In [39]:
# Given the map from part 1, build out the new map for part 2 (5 times wider, 5 times taller, each step out increments all values)
def build_part_2_map(map):
    # Get the new map's size
    mapSize = len(map)
    newMapSize = mapSize * 5
    # Get an array ready for the new map
    newMap = [ [0]*newMapSize for _ in range(newMapSize)] 
    # Iterate through every possible point in the new map array
    for xCoord, _ in enumerate(newMap):
        for yCoord, _ in enumerate(newMap):
            # Grab the value of the equivalent location in the old map
            oldValue = map[xCoord % mapSize][yCoord % mapSize]
            # Update it (increment by 1 for every step down and every step right)
            newValue = oldValue + (1 * int(xCoord / mapSize)) + (1 * int(yCoord / mapSize))
            # If the new value is greater than 9, we decrement it down below 9 again
            if newValue > 9:
                newValue = newValue - 9
            # Update the new map
            newMap[xCoord][yCoord] = newValue    

    return newMap


In [40]:
# Read in the risk map
file = open('puzzleinput.txt')
map = [[int(x) for x in line.strip()] for line in file]
# Get the size of the map to retrieve the end node
mapSize = len(map) - 1
# Build out nodes for each point on the map
nodes = build_nodes(map)

# Solve part 1 (calculate total risk for each node using Djikstra's)
solve(nodes)
# Get the end node and retrieve its total risk
targetNode = nodes.get((mapSize, mapSize))
part1Solution = targetNode.totalRisk

# Build the larger map required for part 2
newMap = build_part_2_map(map)
# Build a new set of nodes based on the new map
nodes = build_nodes(newMap)
# Get the new map size for end node retrieval
mapSize = len(newMap) - 1
# Solve part 2
solve(nodes)
# Get the end node and retrieve its total risk
targetNode = nodes.get((mapSize, mapSize))
part2Solution = targetNode.totalRisk

# Print solution
print("Part 1 solution: The total risk for the map in Part 1 is " + str(part1Solution))
print("Part 2 solution: The total risk for the map in Part 2 is " + str(part2Solution))

Part 1 solution: The total risk for the map in Part 1 is 447
Part 2 solution: The total risk for the map in Part 2 is 2825
